In [0]:
# InceptionNet - made by google specifically for the ImageNet competition
# ImageNet dataset - total number of images are 14,197,122 - no of classes are 1000

In [0]:
# Transfer learning - a technique where we use a pre-trained model for performing custom tasks, like image classification 
# base models - Inception, resnet, mobilenet, vgg16, vgg19, etc. Here, mobilenet used.
# 1. when we define the base model, we delete the few last layers from it that are designed specifically for the imagenet classification. 
# 2. Custom model is then added. The name of this part is called head. 
# 3. base model is frozen and not trained any further. 

# fine tuning - suitable for bigger datasets - sometimes, when the custom dataset is very different from the base model dataset, we need info about it in the base model as well.
# we still add base model and head, but the complete base model is no longer frozen, only the beginning is.

When to use what technique

In [0]:
# if we have a large and dissimilar dataset(than the base model dataset) - train the whole model
# if we have a large and similar dataset - fine tuning
# if we have a small and dissimilar dataset - fine tuning
# if we have a small and similar dataset - transfer learning

In [4]:
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [5]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 114kB/s 
     |████████████████████████████████| 450kB 42.2MB/s 
     |████████████████████████████████| 3.8MB 19.6MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [7]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [8]:
# download dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-03-10 06:20:04--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  80.0MB/s    in 0.8s    

2020-03-10 06:20:05 (80.0 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
import tqdm

#DATA PREPROCESSING

In [0]:
import os
import zipfile # to extract dataset from zip file
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from tqdm import tqdm_notebook # to visualise any progress in the project
from tensorflow.keras.preprocessing.image import ImageDataGenerator # to create automatic image preprocessing pipeline

Unzip the cats and dogs dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")

In [0]:
zip_object.close()

In [0]:
# set up dataset paths
dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

#BUILDING THE MODEL

1. Load the MobileNetV2 base model

In [0]:
IMG_SHAPE = (128,128,3)

In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights="imagenet")
# include_top=False means that head wont be imported.
# weights parameter takes the name of the dataset the base model is trained on.

9412608/9406464 [==============================] - 2s 0us/step


In [18]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

2. Freezing the base model

In [0]:
base_model.trainable = False

3. Adding the custom head to the pre-trained model

In [20]:
# 1. first, check the output shape of the pretrained model. Based on that, we'll decide how many layers,if any, should be added.
base_model.output # input to the custom model

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
# possible options:-

# 1. add flattening layer to convert the output to 1d vector.
# but in this case, dimensions are quite high ,ie, 4 X 4 X 128 and output vector would be too large

# 2. add a global avg pooling layer
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)
# units=1 in the output layer since we're performing binary classification and there are 2 classes

#DEFINING THE MODEL

In [0]:
# instead of Sequential(), here will use simple model since we're combining two models - 
# allows us to specify inputs and outputs separately
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

#COMPILE THE MODEL

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=['accuracy'])
# RMSprop is supposed to be the best optimizer for mobilenet model; and lr value=0.0001 is the best
# we're not directly defining optimizer using 'rmsprop' since we need to change its learning rate. lr should be low for transfer learning
# (since very big models are there)

# CREATING DATA GENERATORS

In [0]:
# tf data generators - used for data preprocessing
# will use 2 image data generators - one for training and one for validation data

In [0]:
# big pre trained architectures support only certain input sizes
# for example : mobilenet supports : (96,96), (128,128),(160,160), (192,192), (224,224)
# we have to reshape our dataset to match these shapes

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_valid = ImageDataGenerator(rescale=1/255.0)

In [38]:
# specify where to find the generator and how to preprocess it
train_generator = data_gen_train.flow_from_directory(train_dir, target_size = (128,128), batch_size=128, class_mode="binary")
# flow_from _directory() loads the specific dataset in the background and therefore, will not occupy any RAM or slow down the 
# training process

Found 2000 images belonging to 2 classes.


In [39]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size = (128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


#TRAINING THE TRANSFER LEARNING MODEL

In [40]:
# when using a data generator, we use a fit_generator() instead of normal fit()
model.fit_generator(train_generator, epochs=5, validation_data = valid_generator)

Epoch 1/5
16/16 [==============================] - 227s 14s/step - loss: 0.4637 - accuracy: 0.8010 - val_loss: 0.4168 - val_accuracy: 0.8380
Epoch 2/5
16/16 [==============================] - 222s 14s/step - loss: 0.4283 - accuracy: 0.8175 - val_loss: 0.3823 - val_accuracy: 0.8630
Epoch 3/5
16/16 [==============================] - 223s 14s/step - loss: 0.4044 - accuracy: 0.8355 - val_loss: 0.3595 - val_accuracy: 0.8660
Epoch 4/5
16/16 [==============================] - 223s 14s/step - loss: 0.3814 - accuracy: 0.8500 - val_loss: 0.3299 - val_accuracy: 0.8840
Epoch 5/5
16/16 [==============================] - 222s 14s/step - loss: 0.3601 - accuracy: 0.8560 - val_loss: 0.3093 - val_accuracy: 0.8920


#TRANSFER LEARNING MODEL EVALUATION

In [41]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
valid_loss, valid_accuracy

(0.31195569410920143, 0.892)

# FINE TUNING

In [0]:
# DO NOT use fine tuning on the whole network, only a few top layers are enough. In most cases, they are more specialized. 
# The goal of fine tuning is to adopt that specific part of the network to our custom (new) dataset.
# Start with fine tuning AFTER completing the transfer learning step. If we try to perform fine tuning immediately, the gradients will 
# be much different between our custom head layer and a few unfrozen layers from the base model.

UNFREEZE A FEW TOP LAYERS FROM THE BASE MODEL

In [0]:
base_model.trainable = True

In [44]:
len(base_model.layers) # no of layers in base model

155

In [0]:
fine_tune_at = 100 # index of the layer after which we'll perform fine tuning - freeze layers before this

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

COMPILE MODEL FOR FINE TUNING

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 229s 14s/step - loss: 0.1549 - accuracy: 0.9430 - val_loss: 0.1132 - val_accuracy: 0.9600
Epoch 2/5
16/16 [==============================] - 225s 14s/step - loss: 0.0220 - accuracy: 0.9990 - val_loss: 0.1125 - val_accuracy: 0.9610
Epoch 3/5
16/16 [==============================] - 224s 14s/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.1119 - val_accuracy: 0.9660
Epoch 4/5
16/16 [==============================] - 223s 14s/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.1409 - val_accuracy: 0.9680
Epoch 5/5
16/16 [==============================] - 225s 14s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1568 - val_accuracy: 0.9670


In [0]:
# for small datasets, it is advisable not to use fine tuning since it may overfit.

In [50]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
valid_loss, valid_accuracy

(0.1572507107630372, 0.967)